### Dice trading

##### 4.a)

$S_t = \{p_t, d_t\}$, avec $p_t$ le nombre de points et $d_t$ le nombre de dés

$a_t = \{0, 1\}$, soit acheter un nouveau dé, soit ne rien faire

$p_t \geq 0$

$d_t \leq 3$

$d_{t+1} = d_t + a_t$

$p_{t+1} = p_t - 5*a_t + \max_{i \in \{1, d_t\}} (u_i (1, 2, 3, 4, 5, 6))$, le nombre de points est déterminé par le score max de tous les dés

##### 4.b) Implement an heuristic

In [46]:
function never_buy_heuristic(s,t) # Always returns "Don't Buy" (never buy dice) 
    return "Don't Buy"
end

never_buy_heuristic_M = zeros(61, 3, 10)


61×3×10 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 

##### 4.c) Write a simulator giving $V_{\pi}$ in a confidence interval of 95%

In [35]:
using Pkg
Pkg.add("Distributions")

   Resolving package versions...
  No Changes to `C:\Users\Ponts\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Ponts\.julia\environments\v1.9\Manifest.toml`


In [36]:
using Distributions
using Random

function simulator_simple_game(heuristic, history=false, T=10)
    nb_dice = 1
    total_points = 0
    println("test")
    if history
        history_list = []
    end

    for turn in 1:T
        action = heuristic((total_points, nb_dice), turn)
        @assert action != "Buy" || total_points >= 6 "RuleError You don't have enough points to buy a dice"
        if action == "Buy" && total_points >= 6 # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
            nb_dice += 1
            total_points -= 5
        end
        dice_roll = maximum(rand(1:6, nb_dice))
        total_points += dice_roll

        if history
            push!(history_list, (action, dice_roll, total_points))
        end
    end
    
    if history
        return total_points, history_list
    end
    return total_points
end
simulator_simple_game(never_buy_heuristic, true, 10)

test


(29, Any[("Don't Buy", 4, 4), ("Don't Buy", 2, 6), ("Don't Buy", 3, 9), ("Don't Buy", 1, 10), ("Don't Buy", 3, 13), ("Don't Buy", 2, 15), ("Don't Buy", 5, 20), ("Don't Buy", 4, 24), ("Don't Buy", 3, 27), ("Don't Buy", 2, 29)])

In [47]:
function simulator_simple_game_V(politic, T=10, N= 100)
    # This function returns the mean and the 95% confidence interval of the total points after T turns, averaged over N games
    nb_dice = 1
    total_points = 0
    Liste = []
    for j in 1:N
        total_points = 0
        for turn in 1:T
            action = politic[total_points + 1, nb_dice, turn]
            @assert action != 1 || total_points >= 6 "RuleError You don't have enough points to buy a dice"
            if action == 1 && total_points >= 6 # Note that the rules prevent the player from buying a dice if they have less than 6 points, even though the dice cost is 5 points
                nb_dice += 1
                total_points -= 5
            end
            dice_roll = maximum(rand(1:6, nb_dice))
            total_points += dice_roll
            
        end
        push!(Liste, total_points)
    end
    return mean(Liste), [mean(Liste) - 1.96*std(Liste)/sqrt(N), mean(Liste) + 1.96*std(Liste)/sqrt(N)]
end
simulator_simple_game_V(never_buy_heuristic_M, 10, 100)

(35.73, [34.68509179099189, 36.7749082090081])

In [38]:
# function compute_expected_cost(heuristic, p=0.05):
#     #p is the error level
#     n=0 # Number of game simulated
#     emp_mean=0
#     emp_dev=0

#     d = Normal()  # Crée une distribution normale standard
#     p = 0.95  # Probabilité pour laquelle vous voulez trouver le quantile
#     q = quantile(d, p)  # Calcule le quantile
    
#     while n<1 and 
#     end
# end

##### 4.d)

In [39]:
for i in 1:6
    println(1 * (i)^2 * 3 - 2)
end

1
10
25
46
73
106


In [40]:
function law_max(nb_dices)
    @assert nb_dices > 0 "RuleError You must have at least one dice"
    @assert nb_dices <= 3 "RuleError You can't have more than 3 dices"
    
    if nb_dices == 1
        return DiscreteUniform(1, 6)
    end
    
    if nb_dices == 2
        values = 1:6
        probs = [1/36, 3/36, 5/36, 7/36, 9/36, 11/36]
        return Categorical(probs)
    end

    if nb_dices == 3
        values = 1:6
        probs = [1/216, 7/216, 19/216, 37/216, 61/216, 91/216]
        return Categorical(probs)
    end
end

law_max (generic function with 1 method)

##### 4.e)

In [41]:
Q = zeros(61, 3, 10, 2) #60 points, 3 dices, 10 turns, 2 actions

#Construct the value function for each state at each time
V = zeros(61, 3, 10)

P = zeros(61, 3, 10, 2, 61) #60 points, 3 dices, 10 turns, 2 actions, 60 points 
for p in 0:60
    for d in 1:3
        for t in 1:10
            for a in 0:1
                if d + a <= 3
                    law = law_max(d + a)
                    for p_prime in 0:60
                        P[p + 1, d, t, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                    end
                end
            end
        end
    end
end
policy_markov = zeros(61, 3, 10)

61×3×10 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 ⋮         
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0

[:, :, 4] =
 0.0  0.0  0.0
 0.0  0.0  0.0
 

We now proceed to the dynamic programming

In [75]:
function dp(T)
    Q = zeros(T * 6 + 1, 3, T, 2) #60 points, 3 dices, 10 turns, 2 actions

    #Construct the value function for each state at each time
    V = zeros(T * 6 + 1, 3, T)

    P = zeros(T * 6 + 1, 3, T, 2, T * 6 + 1) #60 points, 3 dices, 10 turns, 2 actions, 60 points 
    for p in 0:T*6
        for d in 1:3
            for t in 1:T
                for a in 0:1
                    if d + a <= 3
                        law = law_max(d + a)
                        for p_prime in 0:T*6
                            P[p + 1, d, t, a + 1, p_prime + 1] = pdf(law, p_prime - p + 5 * a)
                        end
                    end
                end
            end
        end
    end
    policy_markov = zeros(T * 6 + 1, 3, T)
    for t in T:-1:1
        for s in 0:T * 6
            for d in 1:3
                V[s + 1, d, t] = -100000
                for a in 0:1
                    Q[s + 1, d, t, a + 1] = 0
                    if d + a <= 3 #we assure that we can't have more than 3 dices
                        for s_prime in 0:T * 6
                            Q[s + 1, d, t, a + 1] += P[s + 1, d, t, a + 1, s_prime + 1] * (s_prime + 1 + V[s_prime + 1, d + a, t ])
                        end
                        if Q[s + 1, d, t, a + 1] > V[s + 1, d, t]
                            V[s + 1, d, t] = Q[s + 1, d, t, a + 1]
                            policy_markov[s + 1, d, t] = a
                        end
                    end
                end
            end
        end
    end
    policy_markov = round.(Int, policy_markov)
    return policy_markov
end
V[1, 1, 1]

12.499999999999998

### La valeur optimale trouvée par programmation dynamique est la suivante

In [76]:
policy_markov = dp(10)
C = simulator_simple_game_V(policy_markov, 10, 1000)

(49.563, [49.33523876917292, 49.79076123082709])

 #### On est content car elle est mieux que la valeur de l'heuristique

##### 4.g)
A améliorer, je ne sais pas trop comment m'y prendre...

In [77]:
# We will now implement the dynamic programming with descending time horizon to see when it is never intersting to buy a dice
for T in 10:-1:1
    policy_markov = dp(T)
    if policy_markov == never_buy_heuristic_M
        println("The optimal policy is to never buy a dice after $T turns")
        break
    end
end
println(policy_markov)

[0 0 0; 0 0 0; 1 1 0; 1 1 0; 1 1 0; 1 1 0; 1 1 0;;;]
